In [1]:
import os

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

import pandas as pd
import pickle

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import PCA
from sklearn.manifold import Isomap, TSNE, MDS

In [2]:
os.chdir('F:\Data\Malware')
print(os.getcwd())

F:\Data\Malware


In [ ]:
appearance_df = pd.read_csv('Android-API/drebin-215-dataset-5560malware-9476-benign.csv')
print(appearance_df.to_numpy())

In [ ]:
print(appearance_df.columns.values)

In [ ]:
print(appearance_df.loc[:, 'class':])
print(len(appearance_df.loc[:, 'class'].tolist()))

In [4]:
class_labels = appearance_df.loc[:, 'class'].tolist()

In [ ]:
print(appearance_df.loc[:, 'transact':'WRITE_SECURE_SETTINGS'])
print(type(appearance_df.loc[:, 'transact':'WRITE_SECURE_SETTINGS']))

In [5]:
benign_neg_dict = dict()
name_list = list()
for col_name, col_data in appearance_df.loc[:, 'transact':'WRITE_SECURE_SETTINGS'].iteritems():
    name_list.append(col_name)
    b = 0
    s = 0
    num = 0
    for i in range(len(col_data)):
        if col_data[i] == 1:
            num += 1
            if class_labels[i] == 'B':
                b += 1
            else:
                s += 1
    if b == 0:
        sus_ben_ratio = 1
    else:
        sus_ben_ratio = float(s / num)
    if s == 0:
        ben_sus_ratio = 1
    else:
        ben_sus_ratio = float(b / num)
    df = num / len(class_labels)
    benign_neg_dict[col_name] = [b, s, sus_ben_ratio, ben_sus_ratio, num, df]

In [ ]:
print(benign_neg_dict)

In [ ]:
meta_data = pd.DataFrame(benign_neg_dict).T
meta_data.columns = ['benign', 'malware', 'percent_mal', 'percent_ben', 'total_calls', 'document_frequency']
print(meta_data)

In [7]:
meta_data.to_csv('Android-API/android_col_meta.csv')

In [8]:
appearance = appearance_df.loc[:, 'transact':'WRITE_SECURE_SETTINGS'].to_numpy()

In [ ]:
for i in range(appearance.shape[0]):
    for j in range(appearance.shape[1]):
        if type(appearance[i][j]) != int:
            print(type(appearance[i][j]))
            print(appearance[i][j])

In [10]:
for i in range(appearance.shape[0]):
    for j in range(appearance.shape[1]):
        if type(appearance[i][j]) != int:
            if appearance[i][j] == '0': appearance[i][j] = 0
            else: appearance[i][j] = 1

In [11]:
print(appearance.astype(float))
appearance = appearance.astype(float)
print(appearance.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]]
(15036, 215)


In [13]:
perplexity_array = [5, 10, 15, 20, 25, 30, 35, 40, 45]
max_iter = 10000
random_state = 44
embeddings = []
PCA = False
PCA_to = 50
for perp in perplexity_array:
    tsne = TSNE(perplexity=perp, n_iter=max_iter, random_state=random_state)
    xy_coordinates = tsne.fit_transform(appearance, y=class_labels)
    embeddings.append(xy_coordinates)

with open('Android-API/embeddings_iter_10000_stop_300.pickle', 'wb') as out_embeddings:
    pickle.dump(embeddings, out_embeddings)

KeyboardInterrupt: 